# Verify AirFlow is ready to Start DAG

In [ ]:
import boto3

session = boto3.session.Session()
region = session.region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

In [ ]:
%store -r airflow_bucket_name
%store -r airflow_env_name
%store -r team_role_arn

In [ ]:
client = boto3.client('mwaa')
mwaa_status = ""

def get_airflow_check():
    response = client.get_environment(
        Name=airflow_env_name
    )
    mwaa_status = response["Environment"]["Status"]
    return mwaa_status


mwaa_status = get_airflow_check()
if(mwaa_status != ''):
    print('[ERROR] Cannot find MWAA {} .'.format(airflow_env_name))
else:    
    print('Sucess! {} is ready!'.format(airflow_env_name))

# Triggering DAG from MWAA

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}